In [7]:
import pandas as pd

# load pbp data
df = pd.read_parquet('../data/games.parquet')
teams = df['Home_Team'].unique()

# add Elo columns
df["Away_Starting_Elo"] = 0.0
df["Home_Starting_Elo"] = 0.0
df["Away_Ending_Elo"] = 0.0
df["Home_Ending_Elo"] = 0.0

In [ ]:
from elosports.elo import Elo

# create league
league = Elo(k = 20)

for t in teams:
    league.addPlayer(t)

print(teams)

['L.A' 'ANA' 'DET' 'TOR' 'COL' 'NYR' 'OTT' 'T.B' 'NSH' 'MIN' 'EDM' 'CGY'
 'PHX' 'CBJ' 'BUF' 'ATL' 'VAN' 'NYI' 'FLA' 'CHI' 'DAL' 'MTL' 'S.J' 'PIT'
 'BOS' 'WSH' 'PHI' 'CAR' 'STL' 'N.J' 'WPG' 'ARI' 'VGK' 'SEA']


In [ ]:
season = 2007

# run seasons
for row in df.itertuples(index=True):
    # see https://github.com/ddm7018/Elo/blob/master/tutorial/elo_simulations.py
    if row.Season > season:
        for key in league.ratingDict.keys():
            # year that Thrashers moved to Winnipeg, copy Elo
            if key == 'WPG' and season == 2011:
                league.ratingDict['WPG'] = league.ratingDict['ATL']
            league.ratingDict[key] = league.ratingDict[key] - ((league.ratingDict[key] - 1500) * (1/3.))
        season += 1

    df.loc[row.Index, 'Away_Starting_Elo'] = league.ratingDict[row.Away_Team]
    df.loc[row.Index, 'Home_Starting_Elo'] = league.ratingDict[row.Home_Team]

    if row.Away_Score > row.Home_Score:
        league.gameOver(row.Away_Team, row.Home_Team, False)
    else:
        league.gameOver(row.Home_Team, row.Away_Team, True)

    df.loc[row.Index, 'Away_Ending_Elo'] = league.ratingDict[row.Away_Team]
    df.loc[row.Index, 'Home_Ending_Elo'] = league.ratingDict[row.Home_Team]

df.to_parquet('../data/game_elo.parquet', index=False)
df.head()

,Game_Id,Date,Home_Team,Home_Coach,Away_Team,Away_Coach,Period,Home_Score,Away_Score,Season,Away_Starting_Elo,Home_Starting_Elo,Away_Ending_Elo,Home_Ending_Elo
0,20001,2007-09-29,L.A,MARC CRAWFORD,ANA,RANDY CARLYLE,3,4,1,2007,1345.516656,1499.509042,1341.753723,1503.271976
1,20002,2007-09-30,ANA,RANDY CARLYLE,L.A,MARC CRAWFORD,3,4,1,2007,1503.271976,1341.753723,1491.519613,1353.506085
2,20004,2007-10-03,DET,MIKE BABCOCK,ANA,RANDY CARLYLE,5,3,2,2007,1353.506085,1499.571347,1349.601760,1503.475673
3,20005,2007-10-03,TOR,PAUL MAURICE,OTT,JOHN PADDOCK,4,3,4,2007,1497.798448,1547.101973,1511.849406,1533.051016
4,20006,2007-10-03,COL,JOEL QUENNEVILLE,DAL,DAVE TIPPETT,3,4,3,2007,1602.428948,1589.943756,1594.895872,1597.476831
